# LINEAR PIZZA OVEN

---

### Sources:
- S1: Camera that recognizes the topping [ws://streams:9400](ws://streams:9400)
- S2: Oven thermometer [ws://streams:9500](ws://streams:9500)

---

### Import rsplib and initialize the RSPEngine

In [1]:
from rsplib import RSPEngine

In [2]:
jasper = RSPEngine("http://jasper:8181/jasper")

In [3]:
jasper.tasks()

[]

## Define the query

In [6]:
qid = 'burnt'

### Get the ontology with Ontop from mySQL
- Connect to the ontop container via SSH
- Run   
`./ontop materialize --format rdfxml -m input/cooking-mapping.ttl -p input/cooking.properties -o input/cooking-ontology.owl`  
to export the ontology built from the relational database to an rdf file

In [5]:
a=1